<a href="https://colab.research.google.com/github/cedro3/Sketch-Generation-with-Drawing-Process-Guided-by-Vector-Flow-and-Grayscale/blob/main/sketch_movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# セットアップ

In [ ]:
# githubからコードをコピー
!git clone https://github.com/cedro3/Sketch-Generation-with-Drawing-Process-Guided-by-Vector-Flow-and-Grayscale.git
%cd Sketch-Generation-with-Drawing-Process-Guided-by-Vector-Flow-and-Grayscale

## ビデオから静止画を切り出し
・video_file =  でビデオを指定

In [ ]:
import os
import shutil
import cv2

# imagesフォルダーリセット
if os.path.isdir('input'):
    shutil.rmtree('input') 
os.makedirs('input', exist_ok=True)

# ビデオから静止画を切り出す関数 
def video_2_images(video_file= './movie.mp4', 
                   image_dir='./input/', 
                   image_file='%s.png'):
 
    # Initial setting
    i = 0
    interval = 3
    length = 300 # リミッター
    
    cap = cv2.VideoCapture(video_file)
    while(cap.isOpened()):
        flag, frame = cap.read()  
        if flag == False:  
                break
        if i == length*interval:
                break
        if i % interval == 0:    
           cv2.imwrite(image_dir+image_file % str(int(i/interval)).zfill(6), frame)
        i += 1 
    cap.release()  

video_2_images()

## 静止画をスケッチ画に変換

In [ ]:
from draw import *
from PIL import Image
import os

# output フォルダーリセット
if os.path.isdir('output'):
    shutil.rmtree('output') 
os.makedirs('output', exist_ok=True)

files = os.listdir('./input')
files.sort()
num = len(files)
print('total = ', num)

n=10
period=5
for file in files:
  file_name = file
  draw(file_name, n, period)

## モノクロ動画の作成

In [ ]:
# スケッチ画を集約（モノクロ）
import os
import shutil

# final フォルダーリセット
if os.path.isdir('final'):
    shutil.rmtree('final') 
os.makedirs('final', exist_ok=True)

for i in range(num):
  files = os.listdir('./output/'+str(i).zfill(6)+'/process')
  files.sort()
  file = files[-1]
  print(str(i).zfill(6), file)
  shutil.copy('./output/'+str(i).zfill(6)+'/process/'+file, './final/'+str(i).zfill(6)+'.jpg')

In [ ]:
# スケッチ画から動画を作成（モノクロ）
# output.mp4  削除
if os.path.exists('./output.mp4'):
   os.remove('./output.mp4')

!ffmpeg -r 10 -i final/%06d.jpg -vcodec libx264 -pix_fmt yuv420p output.mp4

In [ ]:
# mp4動画の再生再生（モノクロ）
from IPython.display import HTML
from base64 import b64encode

mp4 = open('./output.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="100%" height="100%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

## カラー動画の作成

In [ ]:
# スケッチ画を集約（カラー）
import os
import shutil

# final フォルダーリセット
if os.path.isdir('finalc'):
    shutil.rmtree('finalc') 
os.makedirs('finalc', exist_ok=True)

for i in range(num):
  shutil.copy('./output/'+str(i).zfill(6)+'/result_RGB.jpg', './finalc/'+str(i).zfill(6)+'.jpg')

In [ ]:
# スケッチ画から動画を作成（カラー）
# output.mp4  削除
if os.path.exists('./outputc.mp4'):
   os.remove('./outputc.mp4')

!ffmpeg -r 10 -i finalc/%06d.jpg -vcodec libx264 -pix_fmt yuv420p outputc.mp4

In [ ]:
# mp4動画の再生再生（カラー）
from IPython.display import HTML
from base64 import b64encode

mp4 = open('./outputc.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="100%" height="100%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")